In [1]:
import numpy as np
from sklearn.utils import shuffle
from keras.utils import np_utils

terrains = ('Plano', 'Bordes',
            'Cráter pequeño', 'Cráter profundo', 'Colina', 'Montaña')

x = np.load(f'terrain_data.npy')
y = np.load(f'terrain_data_labels.npy')
y = y-1

x, y = shuffle(x, y)

x_train, x_val, x_test = np.split(
    np.array(x), [int(len(x)*0.7), int(len(x)*0.85)])
y_train, y_val, y_test = np.split(
    np.array(y), [int(len(y)*0.7), int(len(y)*0.85)])

y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
y_test = np_utils.to_categorical(y_test)

n_clases = len(terrains)
n_features = 17

print(np.unique(y, return_counts=True))
print(len(x_train))
print(len(y_train))
print(len(x_val))
print(len(y_val))
print(len(x_test))
print(len(y_test))
print(n_clases)
print(n_features)


2021-11-11 17:13:22.663325: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


(array([0, 1, 2, 3, 4, 5]), array([1369,  197,  192,   96,  361,  246]))
1722
1722
369
369
370
370
6
17


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras import backend

# We want to make sure we start from the start when training our model everytime we run it.
backend.clear_session()


# Define MLP model
model = Sequential()
model.add(Dense(256, input_dim=n_features, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(n_clases, activation='softmax'))


2021-11-11 17:13:25.574568: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-11 17:13:25.588039: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-11 17:13:26.052305: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-11 17:13:26.052386: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Design computeCapability: 7.5
coreClock: 1.215GHz coreCount: 46 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 357.69GiB/s
2021-11-11 17:13:26.052514: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-11-11 17:13:26.057561: I tensorflow/stre

In [3]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               4608      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 23,334
Trainable params: 23,334
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [5]:
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
es = EarlyStopping(monitor='val_acc', mode='min', verbose=1, patience=10)
# Fit model
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y),
                                                  y)
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), class_weight=(
    dict(zip(np.unique(y), class_weights))), epochs=150, batch_size=6)


/home/hivini/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[0 1 2 3 4 5], y=[0 0 0 ... 1 0 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
2021-11-11 17:13:28.782945: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-11 17:13:28.783559: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2208005000 Hz


Epoch 1/150


2021-11-11 17:13:29.602934: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


287/287 [==============================] - 5s 12ms/step - loss: 9.8535 - accuracy: 0.2441 - val_loss: 3.3593 - val_accuracy: 0.2168
Epoch 2/150
287/287 [==============================] - 3s 9ms/step - loss: 5.5007 - accuracy: 0.1828 - val_loss: 2.2408 - val_accuracy: 0.4336
Epoch 3/150
287/287 [==============================] - 3s 9ms/step - loss: 2.2052 - accuracy: 0.4113 - val_loss: 1.8739 - val_accuracy: 0.4065
Epoch 4/150
287/287 [==============================] - 3s 10ms/step - loss: 2.4707 - accuracy: 0.4953 - val_loss: 1.6553 - val_accuracy: 0.3930
Epoch 5/150
287/287 [==============================] - 3s 9ms/step - loss: 1.6781 - accuracy: 0.4050 - val_loss: 1.6332 - val_accuracy: 0.3442
Epoch 6/150
287/287 [==============================] - 3s 12ms/step - loss: 1.5401 - accuracy: 0.4151 - val_loss: 1.5777 - val_accuracy: 0.4255
Epoch 7/150
287/287 [==============================] - 3s 10ms/step - loss: 1.5153 - accuracy: 0.4290 - val_loss: 1.8436 - val_accuracy: 0.3902
Epoch 8

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_acc)


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
# bo is for blue dot.
plt.plot(epochs, loss, 'g', label='Training loss')
# b is for solid blue line
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
plt.clf()

plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
